In [1]:
import pandas as pd
import gc

train_df = pd.read_parquet('otto/train.parquet')

In [2]:
df_sess = train_df.groupby("session")["aid"].apply(list)
del train_df
gc.collect()

aid_list = []
for aid in df_sess:
    s = ' '.join(str(x) for x in aid)
    aid_list.append(s)

df_sess_split = [sentence.split() for sentence in aid_list]
del aid_list
gc.collect()

df_sess_split

[['1517085',
  '1563459',
  '1309446',
  '16246',
  '1781822',
  '1152674',
  '1649869',
  '461689',
  '305831',
  '461689',
  '362233',
  '1649869',
  '1649869',
  '984597',
  '1649869',
  '803544',
  '1110941',
  '1190046',
  '1760685',
  '631008',
  '461689',
  '1190046',
  '1650637',
  '313546',
  '1650637',
  '979517',
  '351157',
  '1062149',
  '1157384',
  '1841388',
  '1469630',
  '305831',
  '1110548',
  '1110548',
  '305831',
  '1650114',
  '1604396',
  '1009750',
  '1800933',
  '495779',
  '394655',
  '495779',
  '789245',
  '789245',
  '366890',
  '361317',
  '1700164',
  '1755597',
  '789245',
  '784978',
  '1171505',
  '784978',
  '1700164',
  '784978',
  '1521766',
  '1725503',
  '528847',
  '1816325',
  '984597',
  '1072782',
  '173702',
  '1072782',
  '1407538',
  '1629651',
  '1768568',
  '1318324',
  '1840418',
  '1813509',
  '1813509',
  '667924',
  '1226444',
  '709550',
  '709417',
  '1225559',
  '1048044',
  '1052813',
  '1225559',
  '240346',
  '1582117',
  '170

# TF-IDF

In [6]:
!pip install --upgrade pip
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.1 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 22.1.2
    Uninstalling pip-22.1.2:
      Successfully uninstalled pip-22.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 64.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 14.7 MB/s eta 0:00:00


In [7]:
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary


dct = Dictionary(df_sess_split)  # fit dictionary
print(dct)
corpus = [dct.doc2bow(line) for line in df_sess_split]

tfidf_model = TfidfModel(corpus)  # fit model

Dictionary<1855603 unique tokens: ['1009750', '102416', '10268', '1048044', '1048797']...>


In [8]:
dct.save("trim_train_df.dic")
tfidf_model.save("tfidf_model.model")

In [ ]:
# 如果跑到這邊memory不夠的話，可以重開notebook，再用下面這段load回跑好的dct, tfidf_model

#from gensim.corpora import Dictionary
#from gensim.models import TfidfModel

#dct = Dictionary.load("trim_train_df.dic")
#tfidf_model = TfidfModel.load("tfidf_model.model")

# Test df

In [ ]:
test_df = pd.read_parquet('otto/test.parquet')

In [2]:
df_sess_test = test_df.groupby("session")["aid"].apply(list)
del test_df
gc.collect()

aid_list_test = []
for aid in df_sess_test:
    s = ' '.join(str(x) for x in aid)
    aid_list_test.append(s)

df_sess_split_test = [sentence.split() for sentence in aid_list_test]

del aid_list_test
gc.collect()

0

In [7]:
# transform the test set with the tfidf model fitted by training df
corpus_test = [dct.doc2bow(line) for line in df_sess_split_test]

In [12]:
import numpy as np
from operator import itemgetter #https://stackoverflow.com/questions/4174941/how-to-sort-a-list-of-lists-by-a-specific-index-of-the-inner-list



sorted_aid = []

for session_id, index_score_pair in zip(df_sess_test.index, tfidf_model[corpus_test]):
    #print('session:', session_id)
    #print([[dct[id], np.around(freq,decimals=3)] for id, freq in index_score_pair])
    #print(sorted(session, key=itemgetter(1), reverse=True))

    # Sorted aids by tfidf scores
    sorted_aid.append([dct[i[0]] for i in sorted(index_score_pair, key=itemgetter(1), reverse=True)])


# Organize data for submission

In [14]:
import pandas as pd

result_df = pd.DataFrame(zip(df_sess_test.index, sorted_aid),columns =['session', 'aid'])

In [15]:

#result_df['aid_len'] = result_df['aid'].apply(len)

# Find the top 20 popular aids
count_df = pd.DataFrame(train_df['aid'].value_counts())
count_df.columns = count_df.columns.str.replace('aid', 'cnt')
popular_aid = list(count_df.index['cnt'][:20])

popular_aids = []
for i in range(1671803):
    popular_aids.append(popular_aid)


In [16]:
result_df['popular_aids'] = popular_aids
result_df['concat_aid'] = result_df['aid'] + result_df['popular_aids']
result_df['top_20'] = result_df['concat_aid'].apply(lambda x:x[:20])
result_df['submit_aid'] = result_df['top_20'].apply(lambda x: ' '.join(str(s) for s in x))

In [17]:
result_df[['session','submit_aid']]

,session,submit_aid
0,12899779,59625 129004 126836 118524 113279 105091 91325...
1,12899780,1142000 973453 582732 736515 129004 126836 118...
2,12899781,199008 194067 918667 141736 57315 129004 12683...
3,12899782,779477 1494780 413962 595994 834354 476063 562...
4,12899783,255297 300127 1817895 198385 1729553 607638 11...
...,...,...
1671798,14571577,1141710 129004 126836 118524 113279 105091 913...
1671799,14571578,519105 129004 126836 118524 113279 105091 9132...
1671800,14571579,739876 129004 126836 118524 113279 105091 9132...
1671801,14571580,202353 129004 126836 118524 113279 105091 9132...


In [18]:
click_df = result_df[['session','submit_aid']]
click_df['session_type'] = click_df['session'].apply(lambda x: str(x)+'_clicks')

cart_df = result_df[['session','submit_aid']]
cart_df['session_type'] = cart_df['session'].apply(lambda x: str(x)+'_carts')

order_df = result_df[['session','submit_aid']]
order_df['session_type'] = order_df['session'].apply(lambda x: str(x)+'_orders')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [19]:
submit_df = pd.concat([click_df, cart_df, order_df])
submit_df = submit_df[['session_type', 'submit_aid']]
submit_df.rename(columns = {'submit_aid':'labels'}, inplace = True)
submit_df = submit_df.sort_values(by=['session_type'])
submit_df

,session_type,labels
0,12899779_carts,59625 129004 126836 118524 113279 105091 91325...
0,12899779_clicks,59625 129004 126836 118524 113279 105091 91325...
0,12899779_orders,59625 129004 126836 118524 113279 105091 91325...
1,12899780_carts,1142000 973453 582732 736515 129004 126836 118...
1,12899780_clicks,1142000 973453 582732 736515 129004 126836 118...
...,...,...
1671801,14571580_clicks,202353 129004 126836 118524 113279 105091 9132...
1671801,14571580_orders,202353 129004 126836 118524 113279 105091 9132...
1671802,14571581_carts,1100210 129004 126836 118524 113279 105091 913...
1671802,14571581_clicks,1100210 129004 126836 118524 113279 105091 913...


In [20]:
submit_df.to_csv('submisstion.csv',index=False)  